# Summary

Симуляция работы бандитов для оптимизации кликов на рекламных банерах

Лучше всего себя показал TS бандит (regret=1471.230717361168, regret_per_pound=0.00735615358680584)

Странно, но очень плохо сработал UCB1, возможно, есть ошибка в формуле?

In [27]:
import numpy as np
import pandas as pd
import time

from functools import partial
from scipy.stats import randint, uniform, bernoulli
from sim_lib import simulation

pd.options.mode.chained_assignment = None


# eps-greedy

In [5]:
def eps_greedy(history: pd.DataFrame, eps: float):
    if uniform.rvs() < eps:
        n = history.shape[0]
        return history.index[randint.rvs(0, n)]

    ctr = history['clicks'] / (history['impressions'] + 10)
    n = np.argmax(ctr)
    return history.index[n]

policy = partial(eps_greedy, eps=0.08)

In [6]:
# seed for homework
np.random.seed(seed=384758917)

start = time.time()
output = simulation(policy, n=200000)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


1195.28338265419

In [7]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(2792.237649427154, 0.01396118824713577, 174)

In [8]:
output['history']

,impressions,clicks,lifetime,p
132,19843.0,1332.0,10870.812904,0.064972
162,154.0,2.0,18594.827945,0.017514
163,86.0,0.0,5153.010937,0.000849
164,68.0,2.0,5092.571727,0.041281
166,48.0,0.0,5340.552070,0.007253
167,33.0,0.0,1474.181162,0.033849
168,23.0,0.0,4900.260295,0.027273
169,14.0,0.0,5007.022458,0.030857
170,18.0,0.0,8920.324215,0.034653
171,14.0,0.0,1080.025985,0.012549


# UCB1

In [116]:
def ucb1_policy(history: pd.DataFrame):
    upper_bound =  np.where(history['impressions']>0, history['clicks'] / history['impressions'], 1)
    n = np.argmax(upper_bound + np.sqrt(2 * np.log(history['impressions'].sum() + 1) / (history['impressions'] + 1)))
    return history.index[n]

policy = ucb1_policy

In [117]:
# seed for homework
np.random.seed(seed=384758917)

start = time.time()
output = simulation(policy, n=200000)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


937.6374189853668

In [118]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(8576.572829202747, 0.042882864146013736, 185)

In [119]:
output['history']

,impressions,clicks,lifetime,p
169,436.0,10.0,12570.660923,0.014629
173,992.0,89.0,2590.948318,0.077850
174,641.0,37.0,1630.594595,0.059940
177,1318.0,141.0,1959.841208,0.109108
178,515.0,20.0,18537.925424,0.055495
181,358.0,1.0,660.380946,0.005876
182,444.0,11.0,3253.772281,0.032968
184,367.0,2.0,3025.997858,0.008220


# Thompson sampling

In [65]:
def thompson_sampling_policy(history: pd.DataFrame):
    success = history['clicks']
    all_trials = history['impressions']
    probs = zip(success, all_trials-success)
    bandits = [np.random.beta(a + 1, b + 1) for a, b in probs]
    n = np.argmax(bandits)
    return history.index[n]

policy = thompson_sampling_policy

In [66]:
# seed for homework
np.random.seed(seed=384758917)

start = time.time()
output = simulation(policy, n=200000)
end = time.time()
end - start

1 impressions have been simulated
10001 impressions have been simulated
20001 impressions have been simulated
30001 impressions have been simulated
40001 impressions have been simulated
50001 impressions have been simulated
60001 impressions have been simulated
70001 impressions have been simulated
80001 impressions have been simulated
90001 impressions have been simulated
100001 impressions have been simulated
110001 impressions have been simulated
120001 impressions have been simulated
130001 impressions have been simulated
140001 impressions have been simulated
150001 impressions have been simulated
160001 impressions have been simulated
170001 impressions have been simulated
180001 impressions have been simulated
190001 impressions have been simulated


466.23984575271606

In [67]:
# baseline regret
output['regret'], output['regret']/output['rounds'],  output['total_banners']

(1471.230717361168, 0.00735615358680584, 179)

In [68]:
output['history']

,impressions,clicks,lifetime,p
154,156.0,0.0,16595.715595,0.000442
169,3736.0,144.0,11415.767481,0.039232
175,4496.0,596.0,11381.152627,0.137817
178,42.0,1.0,4959.290645,0.030834
